# Political topic unification
---
Testing the merging of political topics into high-level, concise topics.

## Setup

### Import libraries

In [ ]:
from pydantic import BaseModel, Field, field_validator
from openai import OpenAI

In [ ]:
from polids.config import settings

### Set parameters

In [ ]:
system_prompt = """# Role
You are a political data analyst specialized in categorizing policy areas into standardized, concise supercategories for downstream analysis.

# Objective
Process a list of political topics with frequency counts to:
1. Generate concise, high-level policy categories (unified topics).
2. Map each input topic to its corresponding unified category.
3. Output the results as structured JSON following a defined schema.

# Input Format
A list where each line contains a topic and its frequency, separated by |. The list is pre-sorted in descending order by frequency.
```markdown
- Topic String 1 | Frequency 1
- Topic String 2 | Frequency 2
...
- Topic String N | Frequency N
```

# Task
1. **Parse Input:** Extract the topic string and frequency integer from each line of the input markdown list. The topic string alone will be used as the key in the output mapping.
2. **Cluster Semantically:** Group input topics that share core concepts, address related issues, or belong to a clear broader policy domain. Use the descending frequency order as a strong signal for identifying core themes, but prioritize semantic coherence.
3. **Define Unified Topics:** For each cluster, create a single, concise name for the unified topic.
4. **Map Inputs:** Assign each input topic string to exactly one unified topic name.
5. **Generate Output:** Format the results according to the specified JSON schema.

# Guidelines
- **Conciseness:** Unified topic names must be 1-5 words. (e.g., "Economy", "Healthcare", "Climate Action").
- **Mutual Exclusivity:** Unified topics must be distinct and non-overlapping in scope. Each input topic must map to only one unified topic.
- **Comprehensiveness:** All input topics must be mapped to a unified topic.
- **Political Relevance:** Use standard political science or common policy terminology for unified topic names.
- **Balance:** Aim for a reasonable number of unified topics (typically 5-12) – avoid over-granularity or excessive consolidation.

# Example
## Input
- Environmental Protection | 20
- Economic Growth Initiatives | 18
- Immigration reform | 17
- Climate Change Policies | 16
- Healthcare for All | 15
- Job creation programs | 14
- Strengthen military | 13
- Universal Healthcare Coverage | 12
- Improve public schools | 11
- Sustainable Economic Development | 10
- Border security funding | 10
- Renewable energy subsidies | 9
- Lower prescription drug costs | 8
- Reduce corporate tax rate | 7
- Affordable higher education | 6
- Combat Global Warming | 5
- Foreign aid reform | 4
##Output

{
  "unified_topics": [
    "Climate Action",
    "Economy",
    "Immigration & Border Security",
    "Healthcare",
    "Education",
    "Defense & Foreign Policy"
  ],
  "topic_mapping": {
    "Environmental Protection": "Climate Action",
    "Economic Growth Initiatives": "Economy",
    "Immigration reform": "Immigration",
    "Climate Change Policies": "Climate Action",
    "Healthcare for All": "Healthcare",
    "Job creation programs": "Economy",
    "Strengthen military": "Defense & Foreign Policy",
    "Universal Healthcare Coverage": "Healthcare",
    "Improve public schools": "Education",
    "Sustainable Economic Development": "Economy",
    "Border security funding": "Immigration",
    "Renewable energy subsidies": "Climate Action",
    "Lower prescription drug costs": "Healthcare",
    "Reduce corporate tax rate": "Economy",
    "Affordable higher education": "Education",
    "Combat Global Warming": "Climate Action",
    "Foreign aid reform": "Defense & Foreign Policy"
  }
}"""

In [ ]:
max_retries = 5

## Load topics to merge
We're going to use manually defined topics, so as to avoid dependencies on previous steps of the pipeline.

In [ ]:
topics_to_unify = {
    # Social Issues / Civil Rights
    "Protecting LGBTQ+ rights": 18,
    "Ensuring voting access for all": 25,
    "Criminal justice reform initiatives": 22,
    "Addressing systemic racism": 15,
    "Defending Second Amendment freedoms": 28,
    "Common-sense gun safety laws": 26,
    "Protecting freedom of religion": 12,
    "Police funding and accountability": 19,
    "Reproductive healthcare access": 23,
    # Technology & Infrastructure
    "Expanding rural broadband internet": 16,
    "Investing in national infrastructure (roads, bridges)": 30,
    "Cybersecurity for critical systems": 14,
    "Regulating big tech companies": 11,
    "Modernizing the power grid": 17,
    "Funding for scientific research (NIH, NSF)": 9,
    # Government Reform & Ethics
    "Campaign finance reform": 20,
    "Addressing political corruption": 13,
    "Term limits for elected officials": 8,
    "Protecting whistleblowers": 6,
    "Strengthening ethics regulations": 10,
    # Economy & Labor (Different Focus)
    "Raising the federal minimum wage": 24,
    "Supporting small business recovery": 21,
    "Protecting workers' right to organize": 18,
    "Addressing income inequality": 20,
    "Investing in workforce development programs": 15,
    # Agriculture & Environment (Different Focus)
    "Supporting sustainable farming practices": 7,
    "Ensuring food safety and security": 11,
    "Protecting national parks and public lands": 19,
    "Water resource management": 14,
    # Other
    "Addressing the opioid crisis": 22,
    "Improving veterans' healthcare and benefits": 27,
    "Affordable housing and homelessness": 16,
    "Disaster preparedness and relief funding": 10,
}

# Sort the topics by frequency in descending order
sorted_topics = sorted(topics_to_unify.items(), key=lambda x: x[1], reverse=True)
# Format the sorted topics into the required input format
input_markdown = "\n".join([f"- {topic} | {freq}" for topic, freq in sorted_topics])
print(input_markdown)

## Define the output schema

In [ ]:
class UnifiedTopic(BaseModel):
    """
    Represents a high-level policy category with its constituent original topics.
    Ensures each unified topic contains at least two original topics.
    """

    name: str = Field(
        description="Concise name for the unified policy category (1-5 words)",
    )

    original_topics: list[str] = Field(
        description="List of original input topics mapped to this unified category",
    )


class UnifiedTopicsOutput(BaseModel):
    """
    Schema for political topic unification output with nested topic structure.
    Ensures mutual exclusivity through explicit grouping in separate unified topics.
    """

    unified_topics: list[UnifiedTopic] = Field(
        description="List of high-level policy categories with their constituent original topics. "
        "Each unified topic must contain at least two original topics, and all original "
        "topics from input must be included across the list. Categories must be mutually exclusive.",
    )

## Test different LLMs

In [ ]:
# Save the outputs of each method in a dictionary (key = method name)
topics_unification_results = dict()

### Initialize the LLM client

In [ ]:
client = OpenAI(api_key=settings.openai_api_key)

### GPT 4.1 nano

In [ ]:
llm_name = "gpt-4.1-nano-2025-04-14"
completion = client.beta.chat.completions.parse(
    model=llm_name,
    messages=[
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": input_markdown,
        },
    ],
    response_format=UnifiedTopicsOutput,  # Specify the schema for the structured output
    temperature=0,  # Low temperature should lead to less hallucination
    seed=42,  # Fix the seed for reproducibility
)
topics_unification_results[llm_name] = completion.choices[0].message.parsed
assert isinstance(topics_unification_results[llm_name], UnifiedTopicsOutput), (
    "Output does not match the expected schema."
)
topics_unification_results[llm_name]

### GPT 4.1 mini

In [ ]:
llm_name = "gpt-4.1-mini-2025-04-14"
completion = client.beta.chat.completions.parse(
    model=llm_name,
    messages=[
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": input_markdown,
        },
    ],
    response_format=UnifiedTopicsOutput,  # Specify the schema for the structured output
    temperature=0,  # Low temperature should lead to less hallucination
    seed=42,  # Fix the seed for reproducibility
)
topics_unification_results[llm_name] = completion.choices[0].message.parsed
assert isinstance(topics_unification_results[llm_name], UnifiedTopicsOutput), (
    "Output does not match the expected schema."
)
topics_unification_results[llm_name]

### GPT 4.1

In [ ]:
llm_name = "gpt-4.1-2025-04-14"
completion = client.beta.chat.completions.parse(
    model=llm_name,
    messages=[
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": input_markdown,
        },
    ],
    response_format=UnifiedTopicsOutput,  # Specify the schema for the structured output
    temperature=0,  # Low temperature should lead to less hallucination
    seed=42,  # Fix the seed for reproducibility
)
topics_unification_results[llm_name] = completion.choices[0].message.parsed
assert isinstance(topics_unification_results[llm_name], UnifiedTopicsOutput), (
    "Output does not match the expected schema."
)
topics_unification_results[llm_name]

### Compare

In [ ]:
for llm_name, result in topics_unification_results.items():
    print(f"Results from {llm_name}:")
    print(result.model_dump_json(indent=2))
    print("\n")
    print("------------------------\n")

### Implemented solution